In [ ]:

from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import MolsToImage
from rdkit.Chem.Draw import MolToFile
import networkx as nx
import matplotlib.pyplot as plt
import torch
import numpy as np

import pandas as pd
from torch.fx.experimental.migrate_gradual_types.constraint import Transpose

df = pd.read_csv('train.csv')
dt = pd.read_csv('test.csv')


df = df[['IC50_nM', 'Smiles']]

pd.DataFrame(df)



def mol_to_nx(mol):
    G = nx.Graph()

    for atom in mol.GetAtoms():
        G.add_node(atom.GetIdx(),
                   labels=atom.GetSymbol(),
                   atomic_num=atom.GetAtomicNum(),
                   formal_charge=atom.GetFormalCharge(),
                   chiral_tag=atom.GetChiralTag(),
                   hybridization=atom.GetHybridization(),
                   num_explicit_hs=atom.GetNumExplicitHs(),
                   is_aromatic=atom.GetIsAromatic())
    for bond in mol.GetBonds():
        G.add_edge(bond.GetBeginAtomIdx(),
                   bond.GetEndAtomIdx(),
                   bond_type=bond.GetBondType(),
                   bond_type_value=bond.GetBondTypeAsDouble())
    return G

mol = df['Smiles'][0]
mol = Chem.MolFromSmiles(mol)


MolsToImage(mols=[mol], subImgSize=(600,600))


def mol_to_nx(mol):
    G = nx.Graph()

    for atom in mol.GetAtoms():
        G.add_node(atom.GetIdx(),
                   labels=atom.GetSymbol(),
                   atomic_num=atom.GetAtomicNum(),
                   formal_charge=atom.GetFormalCharge(),
                   chiral_tag=atom.GetChiralTag(),
                   hybridization=atom.GetHybridization(),
                   num_explicit_hs=atom.GetNumExplicitHs(),
                   is_aromatic=atom.GetIsAromatic())
    for bond in mol.GetBonds():
        G.add_edge(bond.GetBeginAtomIdx(),
                   bond.GetEndAtomIdx(),
                   bond_type=bond.GetBondType(),
                   bond_type_value=bond.GetBondTypeAsDouble())
    return G



graph_arr = []

count = 0
for i in df['Smiles']:
    mol = Chem.MolFromSmiles(i)
    G = mol_to_nx(mol)
    G = nx.to_numpy_array(G)
    G = torch.tensor(G)
    graph_arr.append(G)
    count  = count + 1

print(graph_arr[0].shape)



각 분자 구조를 그래프 -> 인접행렬 구조로 변환. 각 인접행렬의 크기가 다른 것이 문제임

In [19]:

import torch
from torch import nn, optim
import matplotlib.pyplot as plt
from torch import functional as F

class GNNLayer(torch.nn.Module):
    def __init__(self, dim_in, dim_out):
        super().__init__()
        
        self.linear = nn.Linear(dim_in, dim_out, bias=False)
 
    def forward(self, x, adjacency):
        
        x = self.linear(x)
        x = torch.sparse.mm(adjacency, x)
        
        return x
    

class GNN(torch.nn.Module):
    def __init__(self, dim_in, dim_out):
        super().__init__()
        self.gnn1 = GNNLayer(dim_in, int(dim_in/2))
        self.gnn2 = GNNLayer(int(dim_in/2), dim_out)
        self.linear = nn.Sequential(nn.Linear(dim_in*dim_out, dim_in*dim_out), nn.ReLU(),
                                    nn.Linear(dim_in*dim_out, dim_in*dim_out), nn.ReLU(),
                                    nn.Linear(dim_in*dim_out, dim_in*dim_out), nn.ReLU(),
                                    nn.Linear(dim_in*dim_out, 1))
 
    def forward(self, x, adjacency):
        h = self.gnn1(x, adjacency)
        h = torch.relu(h)
        h = self.gnn2(h, adjacency)
        h = torch.flatten(h)
        h = self.linear(h)
        
        return h


test = graph_arr[0] + torch.eye(graph_arr[0].shape[0])
graph_arr[0] = graph_arr[0].to(torch.float32)
test = test.to(torch.float32)
gnn = GNN(72, 1)

#print(graph_arr[0])
p = gnn(graph_arr[0],test)


print(p)



tensor([-0.1131], grad_fn=<ViewBackward0>)
